# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB (for the businesses)

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

85% class finish this step
6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. (require subquery)
- Find the lowest rated business and return text of the most recent review(need to use business ID to connect 2 tables).  If multiple business have the same rating, select the restaurant with the least reviews. 

#use id as id for businesse, do not need to set up new sets of unique ids.
#each business has a unique id, reach review has a unique id
#set param dict to : 50


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to thin about your DB schema. What tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your DB tables. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Set up the DB

Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

### Foriegn Keys

BFor this project you will have two tables that you need to link using a foreign key. Below is some information about how to create a foriegn key.  

http://www.mysqltutorial.org/mysql-foreign-key/



### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)

In [2]:
## Connect to DB server on AWS
import mysql.connector
import config


cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd
    
)

print(cnx)

cursor = cnx.cursor()

In [3]:
## Create new DB 

#cursor.execute('CREATE DATABASE yelp')


In [4]:
cnx.database = 'yelp'

In [5]:
# Create a table for the Businesses
create_table_business = """CREATE TABLE businesses(
                businessID varchar(255) PRIMARY KEY,
                name varchar(255),
                rating FLOAT,
                review_count INTEGER,
                price varchar(255)
                );
"""



In [6]:
#already ran
#cursor.execute(create_table_business)

In [7]:
# Create a table for the reviews
#already ran
# create_table_review = """CREATE TABLE reviews (
#                 reviewID varchar(255) PRIMARY KEY,
#                 text_of_review varchar(255),
#                 businessID varchar(255),
#                 FOREIGN KEY (businessID)
#                     REFERENCES businesses(businessID)
#                 );
# """


In [8]:
# already ran
# cursor.execute(create_table_review)

## Part 3 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [9]:
# Write a function to make a call to the yelp API
import json
import requests
business_url = 'https://api.yelp.com/v3/businesses/search'

In [10]:
client_id = 'ed8v7wLt_hE4TBRBsMZEdw' #Your client ID goes here (as a string)

api_key = 'WnAwXMZoPDgkIP5kynJgDvzXW2iQvyObxw-ARTwvPz25MD4k0oP9Jj3bmKsYva9xrbMrH1lbUXLwLxLUJwsXZbS80ZeF1blDEKWQ20aKlGbR4KvIXjT5Yg5VLmLpXXYx'
##Your api key goes here (as a string)

In [12]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
#Bearer comes from Yelp site

In [13]:
# what type of business do you want to search
term = 'spa'

#where do you want to perform this search
location = 'NYC'

# what is your other parameter you want to search against
status = True

url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "open_now" : status
                
            }

In [14]:
# Write a script that combines the three functions above into a single process.
response = requests.get(business_url, headers=headers, params=url_params)

# write a function to parse the API response 
response.text
# so that you can easily insert the data in to the DB
business_data = response.json()
business_data

In [16]:
# Write a function to take your parsed data and insert it into the DB
for key in data.keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    print('\n') #Seperate out data

NameError: name 'data' is not defined

In [18]:
business_data['total']

8400

In [19]:
test_data = business_data['businesses'][9] ##testing one data
test_data

{'id': 'vFIuJD2_gVIf7fVXo89qhw',
 'alias': 'caudalie-spa-café-and-wine-bar-new-york',
 'name': 'Caudalie Spa Café & Wine Bar',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/B9iE-_6FiP241e9KDI5x4A/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/caudalie-spa-caf%C3%A9-and-wine-bar-new-york?adjust_creative=ed8v7wLt_hE4TBRBsMZEdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ed8v7wLt_hE4TBRBsMZEdw',
 'review_count': 17,
 'categories': [{'alias': 'spas', 'title': 'Day Spas'},
  {'alias': 'cosmetics', 'title': 'Cosmetics & Beauty Supply'}],
 'rating': 5.0,
 'coordinates': {'latitude': 40.73973, 'longitude': -74.00774},
 'transactions': [],
 'price': '$$$',
 'location': {'address1': '823 Washington St',
  'address2': '',
  'address3': None,
  'city': 'New York',
  'zip_code': '10014',
  'country': 'US',
  'state': 'NY',
  'display_address': ['823 Washington St', 'New York, NY 10014']},
 'phone': '+12122428466',
 'display_phone': '(212) 242-8466'

In [20]:
#loop through  
def parse_data(data):
    parsed_data = []
    for business in data:
        if 'price' not in business.keys():
            business.setdefault('price', None)
        biz_tuple = (business['id'], 
                     business['name'],  
                     business['rating'], 
                     business['review_count'],
                     business['price']
                     )

        parsed_data.append(biz_tuple)
    return parsed_data

parsed_businesses = parse_data(business_data['businesses'])
parsed_businesses

[('mM1uQFMZNZtrGHZzB1ghMQ', 'Aire Ancient Baths', 4.0, 773, '$$$'),
 ('aobUMZMqeNInArssVHuvng',
  "Orient Retreat Women's Spa and Wellness Center",
  4.5,
  130,
  '$$'),
 ('EDdR1Xd-7v_m4_qs3UY3Yw', 'The Mandarin Spa', 4.5, 126, '$$'),
 ('BqNXOy5WJjIoNTX_Xg2wOg', 'Nuansa Spa', 5.0, 183, '$$$'),
 ('VV8yib_qgXF9hVk1V9AB3g', 'Pure Qi Spa', 4.5, 335, '$$'),
 ('Nu6403eEBOQvI58_ly2p7w', 'Ohm Spa', 4.5, 232, '$$$'),
 ('ZkI3V9xcL5r8Tw5kS0BgRw', 'Shibui', 4.0, 34, '$$$'),
 ('G4cXJFAnUMoX6BkmCNTbug', 'SoJo Spa Club', 3.5, 1473, '$$$'),
 ('j13wY82QR8Vzag4jNUtVSg', 'V Nail studio', 5.0, 2, None),
 ('vFIuJD2_gVIf7fVXo89qhw', 'Caudalie Spa Café & Wine Bar', 5.0, 17, '$$$'),
 ('ClY7IwXYIqMF-GPFuH6e_A', 'Zuyuan', 4.5, 546, '$'),
 ('L2I114qmYgpGrARwhJTd4A', 'King Wellness Spa', 4.5, 122, '$$'),
 ('wLDo9mjgv3zw_vfK1tQ5-A', 'Cynergy Spa', 4.5, 121, '$$$'),
 ('V_lzIYWkUMq8gDrcVy8jpg', 'Yihan Spa', 4.5, 133, '$$'),
 ('xHRYw4lx7O81o8MfRTZgHg', 'Glo Boutique Spa', 5.0, 22, None),
 ('V1Z74XUHv6DepYktYUmDiA', 

In [21]:
business_data['businesses'][0].keys()

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance'])

In [22]:
# Write a function to take your parsed data and insert it into the DB

insert_business = """INSERT INTO businesses (businessID, name, rating, review_count, price) VALUES (%s, %s, %s, %s, %s);"""

In [23]:
#insert all of the records and commit it
cursor.executemany(insert_business, parsed_businesses)


IntegrityError: 1062 (23000): Duplicate entry 'mM1uQFMZNZtrGHZzB1ghMQ' for key 'PRIMARY'

In [ ]:
cnx.commit()

In [ ]:
import mysql.connector 
import config

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.pw
)
cursor = cnx.cursor()
cnx.database = 'yelp'

In [ ]:
def insert_data(cnx, cursor, parsed_data):
    insert_business = """INSERT INTO businesses (businessID, name, rating, review_count, price) VALUES (%s, %s, %s, %s, %s);"""
    cursor.executemany(insert_business, parsed_data)
    cnx.commit()
    return

In [ ]:
# Write a script that combines the three functions above into a single process.

# create a variable  to keep track of which result you are in. 

cur = 21

#set up a while loop to go through and grab the result 
while cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    business_data = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_data = parse_data(business_data)
    
    # use your function to insert your parsed results into the db
    insert_data(cnx, cursor, parsed_data)
    #increment the counter by 50 to move on to the next results
    cur += 20

## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [ ]:
# write a SQL query to pull back all of the business ids 

# you will need these ids to pull back the reviews for each restaurant

#Bearer comes from Yelp site

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews
# what type of business do you want to search
term = 'spa'

#where do you want to perform this search
location = 'NYC'

# what is your other parameter you want to search against


url_params = {
                "locale": term.replace(' ', '+'),   
            }

In [ ]:
# Write a function to parse out the relevant information from the reviews
response = requests.get(reviews_url, headers=headers, params=url_params)

In [ ]:
# write a function to parse the API response 
response.text
# so that you can easily insert the data in to the DB
data1 = response.json()
data1

In [ ]:
# write a function to insert the parsed data into the reviews table
stmt = """INSERT INTO students VALUES (%s, %s, %s, %s, %s)"""

In [ ]:
# combine the functions above into a single script  

## Part 5 -  Write SQL queries that will answer the questions posed. 

Now that your data is in the DB, you can answer the questions. Write SQL querires that will answer the questions. **Do not use Pandas in this lab**

In [ ]:
# create connection


In [ ]:
# execute SQL queries
cursor.execute("SELECT* FROM students")

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    return data

In [ ]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    return parsed_results

In [ ]:
def db_insert(cnx, cursor, parsed results):
    # your code to insert and commit the results
    # I would create the connection and cursor outside of this function and then pass it through

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result. 3000 is the max of my results
while cur < num and cur < 30000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20